## Design and Evaluation of a multi-trip delivery model with truck and drones

In [29]:
# importando bibliotecas
import gurobipy as gp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from random import randrange
from geopy import distance

## Colocando os dados das instancias

In [30]:
# lendo dados das instancias
instancia = pd.read_csv("Instancia.txt", sep = "\t")[0:9]
instancia

,Sigla,N,E
0,0,-23.503987,-46.848168
1,1,-23.442280,-46.884854
2,2,-23.477945,-46.853790
3,3,-23.455576,-46.868401
4,4,-23.483889,-46.842870
5,5,-23.458215,-46.829083
6,6,-23.437693,-46.869704
7,7,-23.490255,-46.858548
8,8,-23.439057,-46.832251


In [31]:
# CRINDO UMA LISTA DE DISTANCIAS ENTRE OS PONTOS
fator_circuito = 2.3
lista_distancias = []
lista_distancias_caminhao = []
lista_localizacao = list(instancia[['N', 'E']].itertuples(index=False, name=None))
for localizacao_origem in lista_localizacao:
    for localizacao_destino in lista_localizacao:
        valor = round(distance.distance(localizacao_origem, localizacao_destino).km,2)
        #print(distance.distance(localizacao_origem, localizacao_destino).km)
        lista_distancias.append(valor)
        lista_distancias_caminhao.append(valor * 2.3)
print('distancias colocadas na lista')
lista_distancias_caminhao

distancias colocadas na lista


[0.0,
 17.916999999999998,
 6.762,
 13.225,
 5.2669999999999995,
 12.488999999999999,
 17.618,
 4.255,
 16.951,
 17.916999999999998,
 0.0,
 11.661,
 5.129,
 14.489999999999998,
 13.730999999999998,
 3.7489999999999997,
 13.684999999999999,
 12.396999999999998,
 6.762,
 11.661,
 0.0,
 6.646999999999999,
 2.9899999999999998,
 7.6819999999999995,
 10.924999999999999,
 3.3349999999999995,
 11.132,
 13.225,
 5.129,
 6.646999999999999,
 0.0,
 9.383999999999999,
 9.269,
 4.577,
 9.131,
 9.475999999999999,
 5.2669999999999995,
 14.489999999999998,
 2.9899999999999998,
 9.383999999999999,
 0.0,
 7.2909999999999995,
 13.339999999999998,
 4.0249999999999995,
 11.684,
 12.488999999999999,
 13.730999999999998,
 7.6819999999999995,
 9.269,
 7.2909999999999995,
 0.0,
 10.879,
 10.695,
 4.944999999999999,
 17.618,
 3.7489999999999997,
 10.924999999999999,
 4.577,
 13.339999999999998,
 10.879,
 0.0,
 13.638999999999998,
 8.809,
 4.255,
 13.684999999999999,
 3.3349999999999995,
 9.131,
 4.02499999999999

In [32]:
lista_distancias

[0.0,
 7.79,
 2.94,
 5.75,
 2.29,
 5.43,
 7.66,
 1.85,
 7.37,
 7.79,
 0.0,
 5.07,
 2.23,
 6.3,
 5.97,
 1.63,
 5.95,
 5.39,
 2.94,
 5.07,
 0.0,
 2.89,
 1.3,
 3.34,
 4.75,
 1.45,
 4.84,
 5.75,
 2.23,
 2.89,
 0.0,
 4.08,
 4.03,
 1.99,
 3.97,
 4.12,
 2.29,
 6.3,
 1.3,
 4.08,
 0.0,
 3.17,
 5.8,
 1.75,
 5.08,
 5.43,
 5.97,
 3.34,
 4.03,
 3.17,
 0.0,
 4.73,
 4.65,
 2.15,
 7.66,
 1.63,
 4.75,
 1.99,
 5.8,
 4.73,
 0.0,
 5.93,
 3.83,
 1.85,
 5.95,
 1.45,
 3.97,
 1.75,
 4.65,
 5.93,
 0.0,
 6.27,
 7.37,
 5.39,
 4.84,
 4.12,
 5.08,
 2.15,
 3.83,
 6.27,
 0.0]

In [33]:
## criando um vetor de distancia para o drone
lista_localizacao_uav = []
for localizacao in lista_distancias:
    lista_localizacao_uav.append(round(localizacao * 1,2))
lista_localizacao_uav

[0.0,
 7.79,
 2.94,
 5.75,
 2.29,
 5.43,
 7.66,
 1.85,
 7.37,
 7.79,
 0.0,
 5.07,
 2.23,
 6.3,
 5.97,
 1.63,
 5.95,
 5.39,
 2.94,
 5.07,
 0.0,
 2.89,
 1.3,
 3.34,
 4.75,
 1.45,
 4.84,
 5.75,
 2.23,
 2.89,
 0.0,
 4.08,
 4.03,
 1.99,
 3.97,
 4.12,
 2.29,
 6.3,
 1.3,
 4.08,
 0.0,
 3.17,
 5.8,
 1.75,
 5.08,
 5.43,
 5.97,
 3.34,
 4.03,
 3.17,
 0.0,
 4.73,
 4.65,
 2.15,
 7.66,
 1.63,
 4.75,
 1.99,
 5.8,
 4.73,
 0.0,
 5.93,
 3.83,
 1.85,
 5.95,
 1.45,
 3.97,
 1.75,
 4.65,
 5.93,
 0.0,
 6.27,
 7.37,
 5.39,
 4.84,
 4.12,
 5.08,
 2.15,
 3.83,
 6.27,
 0.0]

In [34]:
lista_1 = []
lista_2 = []
# criando duas lista com os pontos de origem e destino
for pontos in range(len(instancia + 1)):
    for pontos2 in range(len(instancia + 1)):
        lista_1.append(pontos)
        lista_2.append(pontos2)

In [35]:
tupla_or_dest = list(zip(lista_1, lista_2))
tupla_or_dest

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (0, 6),
 (0, 7),
 (0, 8),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (1, 6),
 (1, 7),
 (1, 8),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 4),
 (2, 5),
 (2, 6),
 (2, 7),
 (2, 8),
 (3, 0),
 (3, 1),
 (3, 2),
 (3, 3),
 (3, 4),
 (3, 5),
 (3, 6),
 (3, 7),
 (3, 8),
 (4, 0),
 (4, 1),
 (4, 2),
 (4, 3),
 (4, 4),
 (4, 5),
 (4, 6),
 (4, 7),
 (4, 8),
 (5, 0),
 (5, 1),
 (5, 2),
 (5, 3),
 (5, 4),
 (5, 5),
 (5, 6),
 (5, 7),
 (5, 8),
 (6, 0),
 (6, 1),
 (6, 2),
 (6, 3),
 (6, 4),
 (6, 5),
 (6, 6),
 (6, 7),
 (6, 8),
 (7, 0),
 (7, 1),
 (7, 2),
 (7, 3),
 (7, 4),
 (7, 5),
 (7, 6),
 (7, 7),
 (7, 8),
 (8, 0),
 (8, 1),
 (8, 2),
 (8, 3),
 (8, 4),
 (8, 5),
 (8, 6),
 (8, 7),
 (8, 8)]

In [36]:
zip_iterator = zip(tupla_or_dest, lista_distancias_caminhao) #Get pairs of elements.
zip_iterator2= zip(tupla_or_dest, lista_localizacao_uav)
d_ijnovo= dict(zip_iterator) # distancia do caminhao
d_ij_novo = dict(zip_iterator2)  # distancia do drone

## Modelando o problema novamente com os dados reais

In [37]:
# parâmetros do problema
K = 4 # quantidade de drones
c = 8 # quantidade de clientes
L = 23 # limite de alcance do drone
M = 9999 # um número muito grande
Q = c # máximo número de lançamentos
d_ij = d_ijnovo.copy() #	Travel time between node i and j by truck
d_ij_= d_ij_novo.copy()#	Travel time between node i and j by UAV 


V = range(c+1)#	Set of all nodes in the network. Elements 0 and c + 1 both represent the depot 
V_ = range(1, c+1)#	Set of customers. C = {1, 2, ..., c} |C|= c. 
U = range(K)#	Set of UAVs. |U|= K 
R = range(Q)#	Set of UAV trips

In [38]:
# modelo
m1 = gp.Model("Modelo_final_v1")

# Variáveis de decisão
x_ij = m1.addVars(V, V, vtype = gp.GRB.BINARY) # Binary variable; 1, if truck uses arc (i, j) to move from node i to node j; otherwise, 0.
y_i = m1.addVars(V, vtype = gp.GRB.BINARY) # Binary variable; 1, if truck stops at node i to launch at least one UAV; otherwise, 0.
f_ijur = m1.addVars(V,V_, R,V, vtype = gp.GRB.BINARY)  # Binary variable; 1, if truck stops at node i and launches UAV u to visit customer j in its rth trip from node i; otherwise, 0.
z_i = m1.addVars(V) #Wait time of truck at node i
t_i = m1.addVars(V) # Waiting time of customer i served by truck
t_i_ = m1.addVars(V_) # Waiting time of customer i served by UAV

In [39]:
m1.setObjective(
    gp.quicksum(t_i[i] for i in V_) + gp.quicksum(t_i_[i] for i in V_)
, sense = gp.GRB.MINIMIZE)

In [40]:
c0 = m1.addConstrs(x_ij[i,j] ==
 0 for i in V for j in V if i == j)

c0_= m1.addConstrs(f_ijur[i,j,u,r] == 0 for i in V for j in V_ for u in U for r in R if i == j)
 
c1 = m1.addConstrs(
   gp.quicksum(x_ij[i,j] for j in V) == gp.quicksum(x_ij[j,i] for j in V) for i in V)    

c2 = m1.addConstrs(
    gp.quicksum(x_ij[i,j] for j in V_) <= 1 for i in V)

c3 = m1.addConstrs(
        gp.quicksum(f_ijur[i,j,u,r] for j in V_ for u in U for r in R) <= M * y_i[i] for i in V)

c4 = m1.addConstrs(
    gp.quicksum(x_ij[i,j] for i in V) >= y_i[j] for j in V
)

c5 = m1.addConstrs(t_i[i] + z_i[i] + d_ij[i,j] - (1 - x_ij[i,j]) * M <= t_i[j] for i in V for j in V_ if i != j)


c6 = m1.addConstrs(
    gp.quicksum((d_ij_[i,j] + d_ij_[j,i]) * f_ijur[i,j,u,r]  for j in V_ for r in R) <= z_i[i] for i in V for u in U 
)

c6_ = m1.addConstrs( z_i[i] <= M * gp.quicksum(f_ijur[i,j,u,r]  for j in V_ for u in U for r in R) for i in V
)

c7 = m1.addConstrs(
    gp.quicksum(f_ijur[i,j,u,r] for j in V_) >= gp.quicksum(f_ijur[i,j,u,r+1] for j in V_) for i in V for u in U for r in R [0:-1]
)

c8 = m1.addConstrs(
    gp.quicksum(f_ijur[i,j,u,r] for i in V for u in U for r in R) + gp.quicksum(x_ij[i,j] for i in V) == 1 for j in V_
)

c9 = m1.addConstrs(
    gp.quicksum(f_ijur[i,j,u,r] for j in V_) <= 1 for i in V for u in U for r in R
)

c10 = m1.addConstrs(t_i[i] + d_ij_[i,j] - (1 - f_ijur[i,j,u,r]) * M +
    gp.quicksum((d_ij_[i,j]+ d_ij_[j,i])* f_ijur[i,j_,u,r_] for j_ in V_ for r_ in R if r_ < r) <= t_i_[j] for i in V for j in V_ for u in U for r in R if i != j)
                                                                                                                                                        


c11 = m1.addConstrs( (d_ij_[i,j] + d_ij_[j,i]) * f_ijur[i,j,u,r] <= L for i in V for j in V_ for u in U for r in R )


In [41]:
# limita o tempo de execução
#m1.Params.timeLimit = 3600
# otimiza o modelo
m1.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 5310 rows, 5300 columns and 81785 nonzeros
Model fingerprint: 0x01830458
Variable types: 26 continuous, 5274 integer (5274 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
Presolve removed 2481 rows and 3034 columns
Presolve time: 0.53s
Presolved: 2829 rows, 2266 columns, 38481 nonzeros
Variable types: 16 continuous, 2250 integer (2129 binary)

Root relaxation: objective 0.000000e+00, 888 iterations, 0.17 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   24          -    0.00000      -     -    1s
     0     0    0.00000    0   40          -    0.00000      -     -    1s
     0   

In [42]:
if m1.SolCount == 0:
    print('Sem solucao, status = %d' % m1.Status)
else:
    print('Função objetivo é %.4f\n' % m1.ObjVal)
    
print('Tempo de execucao é %.4fs\n' %m1.Runtime)
print('Resultados\n')
print("Viagens realizadas por caminhão")
for i in V:
    for j in V_:
        if x_ij[i,j].X>0.9:
            print('(%d, %d) - X%d%d = %d' % (i, j, i, j, x_ij[i,j].X))

print("\nViagens realizadas por drone")            
for i in V:
    for j in V_:
        for u in U:
            for r in R:
                if f_ijur[i,j,u,r].X > 0.9:
                    print ('(%d, %d, %d, %d) por Drone #%d - Y%d%d%d%d = %d' % (i, j, u, r, u, i, j, u, r, f_ijur[i,j,u,r].X))


print("\nTempo de espera do caminhão nos nós")
for i in V_:
        if z_i[i].X>0.9:
            print('\n(%d) Foi o tempo de espera do caminhão no nó %d' % (z_i[i].X, i))


print("\nOnde os drones são lançados:")


for i in V:
    if y_i[i].X > 0.9:
        print("O caminhão lançou drones no nó %d: %d" % (i, y_i[i].X))



Função objetivo é 78.3500

Tempo de execucao é 894.9441s

Resultados

Viagens realizadas por caminhão
(0, 7) - X07 = 1

Viagens realizadas por drone
(7, 1, 0, 0) por Drone #0 - Y7100 = 1
(7, 2, 0, 1) por Drone #0 - Y7201 = 1
(7, 3, 2, 1) por Drone #2 - Y7321 = 1
(7, 4, 1, 1) por Drone #1 - Y7411 = 1
(7, 5, 3, 0) por Drone #3 - Y7530 = 1
(7, 6, 1, 0) por Drone #1 - Y7610 = 1
(7, 8, 2, 0) por Drone #2 - Y7820 = 1

Tempo de espera do caminhão nos nós

(20) Foi o tempo de espera do caminhão no nó 7

Onde os drones são lançados:
O caminhão lançou drones no nó 0: 1
O caminhão lançou drones no nó 7: 1


In [43]:
m1.write("modelo.lp")